In [3]:
from langchain_groq import ChatGroq

api_key = "your api key"

In [4]:
llm  = ChatGroq(groq_api_key = api_key, model_name = "llama-3.1-8b-instant")

In [7]:
ans = llm.invoke("Write  receipy of maagi ?")
print(ans.content)

I'm assuming you're referring to "Maggi" which is a popular instant noodle brand. Here's a simple recipe to make Maggi noodles with a flavorful twist:

**Ingredients:**

- 1 packet of Maggi noodles
- 2 cups of water
- 1 tablespoon of oil (vegetable or peanut oil)
- 1 small onion, finely chopped
- 2 cloves of garlic, minced
- 1 small tomato, diced
- 1 teaspoon of chili flakes (optional)
- 1 teaspoon of soy sauce
- Salt to taste
- Your choice of garnish (e.g., chopped green onions, grated cheese, or a slice of lemon)

**Instructions:**

1. **Boil the water**: Boil 2 cups of water in a large saucepan.
2. **Cook the noodles**: Add the Maggi noodles to the boiling water and cook according to the package instructions (usually 2-3 minutes). Drain the water and set the noodles aside.
3. **Heat the oil**: In a separate saucepan, heat 1 tablespoon of oil over medium heat.
4. **Sauté the onion and garlic**: Add the chopped onion and minced garlic to the oil and sauté until the onion is translucen

In [8]:
ans = llm.invoke("write python code for sum of 2 number")
print(ans.content)

**Sum of Two Numbers in Python**

Here's a simple Python function that takes two numbers as input and returns their sum.

```python
def sum_of_two_numbers(a, b):
    """
    Returns the sum of two numbers.

    Args:
        a (float): The first number.
        b (float): The second number.

    Returns:
        float: The sum of a and b.
    """
    return a + b

# Example usage:
num1 = 5
num2 = 10

result = sum_of_two_numbers(num1, num2)
print(f"The sum of {num1} and {num2} is: {result}")
```

In this code:

*   We define a function `sum_of_two_numbers` that takes two arguments `a` and `b`.
*   The function returns the sum of `a` and `b` using the `+` operator.
*   We provide an example usage of the function, where we calculate the sum of `5` and `10` and print the result.

**Tips:**

*   Make sure to handle potential errors, such as non-numeric inputs or overflows.
*   You can modify the function to accept a variable number of arguments using the `*args` syntax.
*   Consider adding 

In [10]:
from langchain.document_loaders.csv_loader import CSVLoader

In [11]:
loader = CSVLoader(
        file_path="codejay_chatbot_full_dataset.csv",
        source_column="Question",
        content_columns=["Answer"],
        metadata_columns=["Question"],
        encoding="utf-8",
        autodetect_encoding=True  # allow trying other encodings if UTF-8 fails
    )

In [14]:
data = loader.load()

In [16]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [18]:
# Initialize embeddings using the Hugging Face model
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\jaypr_ughx7yt\Downloads\Codejay_Chatbot\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
embedding = embedder.embed_query("This is a test document.")

In [21]:
embedding[:5]

[-0.038338545709848404,
 0.1234646886587143,
 -0.028642931953072548,
 0.05365271493792534,
 0.008845364674925804]

In [22]:
len(embedding)

384

In [23]:
from langchain.vectorstores import FAISS

In [27]:
# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data, embedding=embedder)

In [28]:
# Save vector database locally
vectordb.save_local("faiss_index")

In [29]:
# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [30]:
rdocs = retriever.get_relevant_documents("what is codejay")

C:\Users\jaypr_ughx7yt\AppData\Local\Temp\ipykernel_29856\4209023829.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rdocs = retriever.get_relevant_documents("what is codejay")


In [31]:
rdocs

[Document(id='33df6ade-e302-4989-ae44-e59b7667f9d8', metadata={'source': 'What is CodeJay?', 'row': 0, 'Question': 'What is CodeJay?'}, page_content='Answer: CodeJay is a YouTube channel that creates informative and project-based videos on data science, machine learning, artificial intelligence, and related technologies.'),
 Document(id='3e945bc2-8bcc-4633-8d7d-7078a823637b', metadata={'source': 'What type of videos does CodeJay make?', 'row': 1, 'Question': 'What type of videos does CodeJay make?'}, page_content='Answer: CodeJay produces videos on data science concepts, end-to-end machine learning projects, deep learning, NLP, and practical implementation tutorials.'),
 Document(id='7043c253-5a7a-4c40-aef5-edaedafd77ba', metadata={'source': 'Where can I find the CodeJay GitHub repository?', 'row': 19, 'Question': 'Where can I find the CodeJay GitHub repository?'}, page_content='Answer: The GitHub link is shared in each video description. You can also find it by searching "CodeJay GitH

Create RetrievalQA chain along with prompt template


In [32]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer. No Preamble.

CONTEXT: {context}

QUESTION: {question}"""

In [33]:
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [34]:
from langchain.chains import RetrievalQA

In [35]:
chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs={"prompt": PROMPT})

In [52]:
result = chain("How we can run our project on local system")
print(result['result'])

Most of our projects can be run using Jupyter Notebook or Google Colab. We provide setup instructions in each video.
